In [1]:
import pandas as pd
import tweepy
import json
from MyOwnTools.keys import twitter


In [2]:
#create connections and API object for further use 
consumer_key = twitter['consumer_key']
consumer_secret = twitter['consumer_secret']
access_token = twitter['access_token']
access_secret = twitter['access_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#pass auth and set the True values for the time-limit parameters of API-class object
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [3]:
api

#### get tweets from search results

In [4]:
#Set a term for search. Here double quotes for restriction to search exact phrase
search_words = '"data science", -filter:retweets' # ' -filter:retweets' says for it's self

In [5]:
    # Get search results
#Set te start date to search
#date_since = '2019-03-20'
number_of_tweets = 3
tweets = tweepy.Cursor(api.search,
              q = search_words,
              lang = "en",
              tweet_mode = 'extended'
              ).items(number_of_tweets)# method '.items' for set the number of tweeets we want to get
              #geocode="41.766823,72.631776"

In [6]:
#check object created
tweets

In [7]:
#itterate over tweets and assign value to statuses
status = [x._json for x in tweets]

#write down data into the file
with open ('example.txt', 'w+') as tl_file:
    for i in range (number_of_tweets):
        json.dump(status[i], tl_file)#write the json data from API object to the file
        tl_file.write('\n')# take a new line for the next entry
       

In [8]:
#get a glance of data
status

[{'created_at': 'Sat Apr 06 16:07:47 +0000 2019',
  'id': 1114560342225559552,
  'id_str': '1114560342225559552',
  'full_text': 'Data Science Central Thursday Digest, March 28\n https://t.co/orZkJRnd5u #DeepLearning #Algorithms #BigData #NeuralNetworks #AI #DeepLearning #DL #DataScience #BigData #NeuralNetworks #SmartTech #ViperIT',
  'truncated': False,
  'display_text_range': [0, 202],
  'entities': {'hashtags': [{'text': 'DeepLearning', 'indices': [72, 85]},
    {'text': 'Algorithms', 'indices': [86, 97]},
    {'text': 'BigData', 'indices': [98, 106]},
    {'text': 'NeuralNetworks', 'indices': [107, 122]},
    {'text': 'AI', 'indices': [123, 126]},
    {'text': 'DeepLearning', 'indices': [127, 140]},
    {'text': 'DL', 'indices': [141, 144]},
    {'text': 'DataScience', 'indices': [145, 157]},
    {'text': 'BigData', 'indices': [158, 166]},
    {'text': 'NeuralNetworks', 'indices': [167, 182]},
    {'text': 'SmartTech', 'indices': [183, 193]},
    {'text': 'ViperIT', 'indices': [19

In [11]:
#create new data frame with desired columns names
columns = ['tweet_id', 'created_at', 'text', 'retweet_count', 'favor_count', 'location', 'user_id', 'user_name']
counts_df = pd.DataFrame(columns = columns)

with open ('example.txt') as fil: #open file
#read file line by line, extract data, store it in dataframe object and append it to counts_df data frame
    for line in fil:
        status = json.loads(line) #get a json object out of each line in a file
        tweet_id = status['id_str'] #get id element
        created_at = status['created_at'] #get timestamp
        retweet_count = status['retweet_count']# get retweets count
        favor_count = status['favorite_count']#get favorites count
        text = status['full_text']
        location = status['user']['location']
        user_id = status['user']['id']
        user_name = status['user']['name']
        counts_df = counts_df.append(pd.DataFrame(\
        [[tweet_id, created_at, text, retweet_count, favor_count, location, user_id, user_name]], columns = columns))
counts_df = counts_df.reset_index(drop = True)
counts_df.shape

(3, 8)

In [12]:
counts_df.head()


,tweet_id,created_at,text,retweet_count,favor_count,location,user_id,user_name
0,1114560342225559552,Sat Apr 06 16:07:47 +0000 2019,"Data Science Central Thursday Digest, March 28...",0,0,"Fortaleza, Brasil",15175317,Daniel Bastos 🇧🇷
1,1114559388986093568,Sat Apr 06 16:04:00 +0000 2019,"GigaOm #Webinar, 4/9: What’s Best for Analytic...",0,1,"Austin, TX",2893971,GigaOm
2,1114559178968829952,Sat Apr 06 16:03:10 +0000 2019,Support the data science community! Nominate a...,0,0,"Boston, MA",3018841323,Open Data Science
